# fasttext基本函数

## 1.1 加载fasttext词语向量并打印fasttext模型的帮助文件

fasttext使用C++写的，官网提供了Python的接口https://github.com/facebookresearch/fastText/tree/master/python 。还有一个使用 fastText的Python库叫做pyfasttext,但是它已经不再维护了https://github.com/vrasneur/pyfasttext 。

In [1]:
from fastText import load_model
fasttext_model = load_model(r'../data/zhwik9_50d.bin')

## 1.2 fasttext模型基本方法演示

In [2]:
# 获取词汇表
words = fasttext_model.get_words()
# 打印词汇表长度
print("词汇表长度=", len(words))
# 打印词汇表前100个词
print("词汇表前100个词\n", words[:100])
# 打印词语向量的维度
print("词语向量维度=",fasttext_model.get_dimension())
# 获取“是”字在词汇表中以及词语向量矩阵中的索引id
print('"是"字在词汇表中以及词语向量矩阵中的索引id', fasttext_model.get_word_id("是"))
# 通过id获取词语"是"向量
vec_by_id = fasttext_model.get_input_vector(5)
# 打印向量长度
import numpy as np
print('词语"是"的向量长度=', np.linalg.norm(vec_by_id))
# fasttext向量库中的词语向量不是单位向量，如果使用余弦相似度查找最相似的词，则需要事先对词语向量进行归一化处理
# 通过词语获取词语“是”的词语向量
vec_by_word = fasttext_model.get_word_vector("是")
print('比较两种获取词语"是"向量的方法是否一致?', np.linalg.norm(vec_by_id) == np.linalg.norm(vec_by_word))

词汇表长度= 388539
词汇表前100个词
 ['</s>', '，', '的', '。', '*', '年', '、', '|', '（', '）', '=', '==', '在', "'''", '是', ':', '：', '日', '為', '.', '-', '一', '和', '(', ')', '===', '有', '》', '《', '國', '人', '中', '||', '了', '/', '於', '大', '會', '後', '「', '」', '學', '時', '與', '之', '被', '1', '上', '以', '2', '也', '而', '他', '第', '及', '“', '”', '不', '個', '由', ',', '軍', '为', '地', '·', '**', '等', '三', '來', '區', '發', '；', '其', '3', '站', '到', '但', '長', '成', '香港', '戰', '名', '對', '灣', '多', '台', '所', '動', '#', '內', '至', '位', '並', '家', '用', '生', '或', '業', '員', '美']
词语向量维度= 50
"是"字在词汇表中以及词语向量矩阵中的索引id 14
词语"是"的向量长度= 22.646149
比较两种获取词语"是"向量的方法是否一致? False


# 2. 计算与"直升机"最相似的词语列表

## 2.1 首先对fasttext词语向量矩阵的每个词向量（行向量）归一化

In [3]:
# 获取整个fasttext向量矩阵
fasttext_vec = fasttext_model.get_input_matrix()[:len(fasttext_model.get_words())]
# 验证读取处的向量矩阵是否正确
input_vec = fasttext_vec[fasttext_model.get_word_id("的")]
model_vec = fasttext_model.get_word_vector("的")
print("input_vec与model_vec一致吗?", np.linalg.norm(input_vec) == np.linalg.norm(model_vec))
# 对fasttext向量矩阵的每个词向量（行向量）归一化
fasttext_vec = (fasttext_vec.T/np.linalg.norm(fasttext_vec,axis=1)).T
# 获取“的”的词语向量
word_vec = fasttext_vec[fasttext_model.get_word_id("的")]
# 输出“的”的向量长度
print(np.linalg.norm(word_vec))

input_vec与model_vec一致吗? False
0.9999999


## 2.2 检索与"直升机"相关的词

In [4]:
# 获取直升机的向量
word_vec = fasttext_vec[fasttext_model.get_word_id("直升机")]
# 计算直升机向量与库中每个词的相似度
sim_vec = np.dot(word_vec, fasttext_vec.T)
# 按相似度排序词语
sorted_sim_vec = sorted(zip(fasttext_model.get_words(), sim_vec), key=lambda x:x[1], reverse=True)
# 获取与直升机最相似的top10词语列表，
print(sorted_sim_vec[:10])
# 把词语直升机自己去除掉
print(sorted_sim_vec[1:10+1])

[('直升机', 1.0), ('运输机', 0.8783453), ('飞机', 0.8404097), ('轰炸机', 0.83333534), ('侦察机', 0.8327074), ('预警机', 0.8304924), ('水上飞机', 0.8284873), ('歼击机', 0.82263386), ('战机', 0.8117472), ('战斗机', 0.8116504)]
[('运输机', 0.8783453), ('飞机', 0.8404097), ('轰炸机', 0.83333534), ('侦察机', 0.8327074), ('预警机', 0.8304924), ('水上飞机', 0.8284873), ('歼击机', 0.82263386), ('战机', 0.8117472), ('战斗机', 0.8116504), ('舰载机', 0.79990345)]
